# Continue Pretraining on T5

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('../../Models/my_tokenizer')

In [2]:
%env WANDB_PROJECT=english-v2

env: WANDB_PROJECT=english-v2


In [3]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration, Adafactor
from accelerate import Accelerator

model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large", device_map='cuda:0')
model.resize_token_embeddings(len(tokenizer))

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=False, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

training_args = TrainingArguments(
    output_dir="buddhist-large-pretrain",
    auto_find_batch_size=True,
    learning_rate=3e-4,
    num_train_epochs=1
    )

In [ ]:
from datasets import load_from_disk
import gc

for i in range(0, 180):
    
    start = i * 10_000
    end = start + 10_000

    print(f'Training on {start} to {end}')

    tokenized_dataset = load_from_disk('../Data/tokenized-pretraining-ds').select(range(start, end))
    gc.collect()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        optimizers=(optimizer, None)
    )

    trainer.train()

    del tokenized_dataset
    gc.collect()

Training on 0 to 50000


Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


In [6]:
model.save_pretrained('../../Models/large-models/pretrained-large-model')